In [2]:
import json
import requests
import os
import pprint
import pandas
import time
from datetime import datetime
from datetime import date
from yaspin import yaspin
pp = pprint.PrettyPrinter()

In [3]:

TFE_ADDR =  os.environ.get('TFE_ADDR', 'https://app.terraform.io')
TFE_API_TOKEN =  os.environ.get('TFE_API_TOKEN', '')
TFE_SITE_ADMIN =  os.environ.get('TFE_SITE_ADMIN', 'false')
TFE_PAGE_COUNT =  os.environ.get('TFE_PAGE_COUNT', 100)
TFE_FILTER_START_DATE = os.environ.get('TFE_FILTER_START_DATE', '01.01.2023')
TFE_FILTER_END_DATE = os.environ.get('TFE_FILTER_END_DATE', date.today().strftime('%d.%m.%Y'))

date_start = datetime.strptime(TFE_FILTER_START_DATE, '%d.%m.%Y')
date_end = datetime.strptime(TFE_FILTER_END_DATE, '%d.%m.%Y')

headers = {
    'Content-Type': 'application/vnd.api+json',
    'Authorization': f"Bearer {TFE_API_TOKEN}"  
    }

In [4]:
@yaspin(text="Loading...")
def getOrgObj():
    result = []
    resp = requests.get(TFE_ADDR+'/api/v2/organizations', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    todo_item = resp.json()['data']
    for item in todo_item:
         info = { 
         'name': item['id'],
         'total_runs': getRunsTotalCount(item['id']),
         'workspace_count': getWorkspaceCount(item['id']),
         'total_applies': getAppliesTotalCount(item['id']),    
         'workspaces': getWorkspaceInfo(item['id'])
         }
         result.append(info)
    return result    

def getRunsTotalCount(organisation):
    total_runs = 0
    todo_item = getWorkspaceInfo(organisation)
    for item in todo_item:
        total_runs +=  getRunsCount(item)
    pp.pprint(f"Total runs for {organisation}: {total_runs}")     
    return total_runs

def getAppliesTotalCount(organisation):
    total_applies = 0
    todo_item = getWorkspaceInfo(organisation)
    for item in todo_item:
        total_applies +=  getAppliesCount(item)
    pp.pprint(f"Total applies for {organisation}: {total_applies}")    
    return total_applies

def getWorkspaceInfo(organisation):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    return retorno


def getWorkspaceCount(organisation):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/organizations/{organisation}/workspaces?page[size]=100&page[number]={page}', headers=headers )
    return len(retorno)

def getRunsCount(workspace_id):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong: {resp}")
        return ""
    todo_item = resp.json()['data']
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            creation_date = datetime.strptime(item['attributes']['created-at'][:10], "%Y-%m-%d")
            if date_start < creation_date and creation_date <= date_end : retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    return len(retorno)
    
def getAppliesCount(workspace_id):
    page = 1
    retorno = []
    resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    if resp.status_code != 200:
        # This means something went wrong.
        pp.pprint(f"something went wrong while counting applies: {resp}")
        return ""
    todo_item = resp.json()['data']
    metdata =   resp.json()['meta']['pagination']
    current_page = metdata["current-page"]
    total_pages = metdata["total-pages"]
    while page <= total_pages:
        todo_item = resp.json()['data']
        metdata =   resp.json()['meta']['pagination']
        current_page = metdata["current-page"]
        for item in todo_item:
            creation_date = datetime.strptime(item['attributes']['created-at'][:10], "%Y-%m-%d")
            if item['attributes']['status'] == "applied" and date_start < creation_date and creation_date <= date_end : retorno.append(item['id'])
        page += 1
        resp = requests.get(TFE_ADDR+f'/api/v2/workspaces/{workspace_id}/runs?page[size]=100&page[number]={page}', headers=headers )
    return len(retorno)    

In [5]:
data = pandas.DataFrame(getOrgObj())
data

⠧ Loading... 'something went wrong: <Response [429]>'


TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [ ]:
df = data[['name','total_runs','total_applies']]
df

In [ ]:
ax = df.head().plot.bar(x='name', y='total_runs', rot=0)

In [ ]:
data

In [ ]:
test = data[['name' == 'devworks']]
test

In [ ]:
data 

In [ ]:
clear